<a href="https://colab.research.google.com/github/huisung/eungdapso_seoul_go_kr_complaints_scraping/blob/master/get_seoul_complaints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 서울시 민원 데이터 가져오기

- 참고사이트 코드를 수정하여 작성
- 참고 : [[파이썬] BS4 서울시 응답소 크롤링](https://systemtrade.tistory.com/350)

In [0]:
import urllib.request
from bs4 import BeautifulSoup
import re

import sys
if 'ipykernel' in sys.modules:
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm

In [0]:
def get_seoul_complaints(pages=1, search_str=None, sdate=None, edate=None):
    list_url = 'http://eungdapso.seoul.go.kr/Shr/Shr01/Shr01_lis.jsp'
    detail_url = 'http://eungdapso.seoul.go.kr/Shr/Shr01/Shr01_vie.jsp'
    
    error_cnt = 0
    rheader = {}
    rtn_list = [('date', 'title', 'contents', 'answer')]
    try:
        for page in tqdm(range(1, pages+1)):
            rheader['page'] = str(page)
            
            if search_str is not None:
                rheader['search_flag'] = 'Y'
                rheader['mSearchGB'] = 'SJCN'
                rheader['mSearchGB_out'] = 'SJCN' # SJ 제목 CN 내용 SJCN 제목+내용
                rheader['SearchCmd'] = ''
                rheader['SearchCmd_out'] = search_str
            if sdate is not None and edate is not None:
                rheader['StartDate_out'] = '-'.join(sdate)
                rheader['StartDate'] = ''.join(sdate)
                rheader['EndDate_out'] = '-'.join(edate)
                rheader['EndDate'] = '-'.join(edate)
        
            request_header = urllib.parse.urlencode (rheader)   
            request_header = request_header.encode ('utf-8')           
            url = urllib.request.Request (list_url, request_header )
            res = urllib.request.urlopen(url). read(). decode('utf-8')    
            
            bs = BeautifulSoup(res , 'html.parser' )
            listbox = bs.find_all ('li', class_='pclist_list_tit42') 
            clist = []   
            for i in listbox :
                clist .append (re.search ('\d{14}', i.find('a')['href']). group() ) 
            
            for p in clist :
                request_header = urllib.parse.urlencode ( {'RCEPT_NO' : str(p) } )    
                request_header = request_header.encode ( 'utf-8' )
            
                url = urllib.request.Request(detail_url, request_header)
                res = urllib.request.urlopen(url). read(). decode('utf-8')
            
                bs = BeautifulSoup(res, 'html.parser')
            
                title = bs.find('th', text = re.compile('제목')).parent.find('td').get_text().strip()
                adate = bs.find('th', text = re.compile('공개일')).parent.find('td').get_text().strip()
                contents = bs.find('p', class_='question_title').parent.get_text()[8:].strip()
                answer = bs.find('p', class_='answer_title').parent.get_text()[8:].strip()
                rtn_list.append((adate, title, contents, answer))
                error_cnt = 0
    except Exception as e:
        print(e)
        rtn_list.append(('error', '', '', ''))
        error_cnt += 1
        if error_cnt == 100:
            rtn_list.append(('error100', '', '', ''))
            return rtn_list
    
    return rtn_list

In [3]:
if __name__ == '__main__' or 'google.colab' in sys.modules:
    print(get_seoul_complaints())
    print(get_seoul_complaints(pages=2, search_str='교통'))
    print(get_seoul_complaints(pages=2, search_str='교통',
                               sdate=['2017', '01', '01'],
                               edate=['2018', '01', '01']))


[('date', 'title', 'contents', 'answer'), ('2019-11-02', '아이와 함께 돌아다니기가 무섭습니다.', '22개월 아이와 아내, 그리고 목줄에 묶인 마티즈 5개월짜리와 함께 산책좀 할겸 나왔습니다. 장승배기에서 노량진역 사이에 도로는 자동차 도로뿐만 아니라 인도에도 오토바이들이 다니고, 지역 자체가 공도가 골목마다 구석구석 있어서 아이와 함께 도보 하기가 어렵습니다.때문에 인근에 노량진 초등학교에 11월 1일 18시 30분정도에 방문해보니보안원의 초소앞에 신분증 제시라고 적혀있어서"여기 운동장 쓰는데도 신분증 드려야하나요?" 라고 물어보니매몰차게 "여기서 나가요" 라고 하시더군요.22살짜리 애도 감정을 느낄수있는데, 아빠를 한번 쳐다보더군요. 아직도 기억에 선합니다. 그래서 아이에게 별일 아닌듯 차량으로 한강 공원으로 나가기로 하고, 당장 나왔습니다. 노량진 초등학교 정문은 골목 끝에 있기때문에 사선으로 처리 되어있습니다.초등학교 우편에 커피숖에서 차한잔 사고 나왔는데,밖에있던 아내와 22개월 아가에게 보안원이 잡상인 취급하듯이 차가 지나다니는곳에 있다고 비키라고 압박했더군요.저도 10년 가까이 운전 했지만, 무리가 될만한 곳에 기다리라는 말은 하지도 않았고혹시나 무리가 되는곳이라면 제가 먼저 이동시켰을겁니다.세상이 흉흉해서 학교에 출입을 제한하는것은 자식키우는 입장에서 충분히 이해할수 있으나 , 시민의 세금으로 운영되는 학교에 이용을 못하는것은 따지고 싶지도 않지만잡상인 취급이나 당하고 아직 말못하는 아기에게 갈곳없는 서울살이를 경험하게 하고 싶지 않습니다. 정말 오토바이 배달업체들의 보험은 사업자가 부담하고 배달 하는 라이더들이 사고시에 부담하는부분이 없으니 인도건 공도건 무섭게 질주하여아이가 갈곳조차 없는 상태에서 이런 부당함은 있을수 없다고 생각합니다.그 노량진 초등학교에 근무하고 있는 보안원을 징계해주시기 바랍니다.', '00님 안녕하십니까?   평소 시정에 많은 관심을 가져주시는 귀하께 감사의 말씀을 드리며


[('date', 'title', 'contents', 'answer'), ('2019-10-31', '세금으로 민노총 가입 광고-즉각 중단해 주십시오.', '지하철과 거리 곳곳에 노조 가입을 독려하는 현수막과 광고 홍보물이 게시되어 있습니다.심지어 공영방송인 교통방송에서도 노조가입 광고를 듣게 됩니다.세금이 특정 세력을 위한 정치적 목적으로 사용된다는 생각으로 매우 불쾌합니다.민노총 가입 홍보 즉각 중단해 주시고아래 질문에 성실한 답변 부탁드립니다.1. 서울시가 왜 민노총 가입을 독려하는 겁니까? 공직자로서 중립성을 지켜야 할 의무는 없습니까?2. 왜 시민의 세금을 민노총이라는 특정 단체 가입 홍보비용으로 지출합니까? 3. 시민의 공적 자원인 지하철에 특정 정치적 성향의 광고 게재는 합법적인 것입니까?4. 공공재인 방송주파수를 사용하고 공적자금으로 운영되는 공영방송인 tbs에서 서울시의 지원을 받아 노조 가입 독려 광고방송을 내보내는 것은, 공영방송의 책임에 벗어나는 것 아닙니까?공영방송은 특정 이익층을 대변하지 않고, 특정 정파나 기관, 업체, 단체에 편항되지 않도록 방송해야 할 의무가 있지 않습니까?왜 시민의 자산인 공영방송을 자신과 정치적 신념이 같은 단체를 지원하는 도구로 사용하십니까?답변 부탁드리며,참고로 교통방송 홈페이지에 게재된 방송윤리 강령 아래에 덧붙입니다.------------------------TBS 방송윤리강령: TBS는 시민의 방송이다.tbs는 특정 이익층을 대변하지 않고 정파적 편향성에서 벗어나 공공성과 공정성, 보편성, 다양성, 정확성, 객관성을 바탕으로 한 프로그램이나 뉴스를 제작, 방송하는 언론의 기본적인 역할을 적극적으로 수행한다.TBS 방송편성 규약제3조 (방송편성의 기준) ①시민의 다양한 의견을 폭넓게 수렴하고 반영하여 사회통합의 역할을 수행하고 시민사회의 기본가치에 대한 인식을 공유하는 프로그램을 적극 방송한다.③방송의 공정성, 독립성, 자율성을 바탕으로 특정 정파나 기관, 업체, 단체에 편향되지 않도록 방송한다.',


[('date', 'title', 'contents', 'answer'), ('2018-01-01', '가재울 9구역 상생스토어타운 조성으로 상권 활성화 제안', '존경하는 서울시장님무술년 새해 복 많이 받으십시오.^^항상 건강하시고 하시는 일마다 대성하시길 진심으로 기원하겠습니다.얼마전 가재울9구역 관리처분 승인 소식을 들었습니다.좋은 소식이지만 가재울뉴타운 주민들의 복지 등 삶의 질이 개선되어져야 한다고 생각합니다.현재 가재울뉴타운 주민들은 교통 및 교육/주거환경 개선에 대해 끊임 없이 의원님 블로그, 서대문구청 민원실, 구청장님 블로그, 더 나아가 구의원 블로그에도 민원 및 의견 개진 활동을하고 있습니다. 교통,교육,복지,주거환경 이슈중 가재울뉴타운 주민들의 삶의 질 개선에 대해 제안드리겠습니다.현재 가재울뉴타운 주민들은 9구역에 대형마트 입점을 적극 희망하고 있습니다. 대형마트 유치는 내년 지방 선거에 많은 영향을 끼칠 것 같습니다.서대문구가 중요시하는 정책에 대해사전 공감이 필요하며 대형마트 입점 가능여부 및 계획에 대해 가재울 뉴타운 주민들 또한 매우 궁금해 합니다.가재울뉴타운 주민으로서 한가지 제안을 드리고 싶습니다.가재울 9구역에 기존 모래내, 서중시장 상인들과 대형유통기업과의 상생스토어 타운 형성 제안을 드립니다.대형유통기업, 소상공인 간 상생/협력 기대효과1. 청년 상인 지원으로 일자리 창출2. 카페, 놀이방, 경로당 등 고객 편의를 위한 부대시설 확충으로 삶의 질 개선3. 전통시장과 공동 마케팅으로 상권 활성화로 지역 경제 발전 기여4. 대형유통업체와 소상공 상인간의 상생/협력 성공사례 사업 전개로 유동인구 유입과 국내/해외 관광객 대상 관광명소로 입지를 다져 서대문구 이미지 제고에 기여ㅇ마트 상생스토어 성공사례https://youtu.be/h4ZDG-EhbrIhttp://www.inthenews.co.kr/mobile/article.html?no=9207http://mediasr.kr/archives/38066', '안녕하십니까?시정발전

## 사용예

In [4]:
import pandas as pd

rtn = get_seoul_complaints(pages=30, search_str='교통')
pd.DataFrame(rtn[1:], columns=rtn[0])

,date,title,contents,answer
0,2019-10-31,세금으로 민노총 가입 광고-즉각 중단해 주십시오.,지하철과 거리 곳곳에 노조 가입을 독려하는 현수막과 광고 홍보물이 게시되어 있습니다...,안녕하십니까. 귀하께서 120다산콜센터를 통해 접수하신 민원(접수번호 2019103...
1,2019-10-31,서울에 오지 강북구,"서울에 오지 강북구에도 회망을 주십시오..발전도 없고,교통도 최악..서울 주요...","안녕하십니까?우리시 시정에 깊은 관심을 가지고 좋은 의견을 주신 점에 감사드리며, ..."
2,2019-10-19,보기 좋은 도시 / 살기 좋은 도시,<<<박원순 시장님께는 첨부된 편지로 꼭 전달해 주세요오늘 10월 18일 광화문광장...,000 님 안녕하십니까.광화문광장 사업을 비롯한 서울시 주요 정책에 관심가지고 좋은...
3,2019-10-14,서울시 자전거도로의 지향점이 무엇인가요.,자전거도로의 비젼은 제3의 교통도로 아닌가요?그래서 이런저런 모빌리티도 받아들이고 ...,"000 님 안녕하세요?먼저 우리 시정에 깊은 관심을 가져주신 점에 대해 감사드리며,..."
4,2019-10-13,"암사생태, 고덕수변공원 및 강일지구 한강공원 관련 제안","암사생태, 고덕수변공원 및 강일지구 한강공원 관련 제안(첨부 있습니다.)안녕하세요,...",서울시정에 많은 관심과 의견을 주셔서 감사드립니다. 안00님께서 제안해 주신 사항에...
...,...,...,...,...
295,2018-01-29,영어마을 입소시 셔틀버스 지원 관련,사회배려계층으로 손자가 기회가 될 때마다 방학을 이용하여 서울시에서 제공하는 사회배...,000님 안녕하세요. 우리시에서 추진하고 있는 영어마을 사업에 많은 관심과 애정을 ...
296,2018-01-29,목동1.2.3단지 3종맞습니다!,종세분화과정에서 목동14개단지중 왜 1.2.3단지만 2종으로 이해가 안됩니다! 매뉴...,1. 시정발전에 깊은 관심을 가지고 협조하여 주시는 000님께 깊은 감사의 말씀을 ...
297,2018-01-29,목동1~3단지 종복원,종세분화과정에서 왜 1.2.3단지만 2종으로 분류하였는지이해가 안됩니다! 고층비율로...,1. 시정발전에 깊은 관심을 가지고 협조하여 주시는 000님께 깊은 감사의 말씀을 ...
298,2018-01-28,자동차 미세먼지 및 시민 인명 피해 발생 고발함,1 측정장비 2대 연구소배치 보고 내용 거짓보고 전국책연구소 전수조사 환경단체확인2...,000님 안녕하십니까? 매연저감장치로 인하여 여러 가지 문제점 해결을 위하여 좋은...
